In [1]:
Output = ('/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1011. Emission Mixtures/P1011.3. Analyses/P1011.3.1. Data Processing/Output')
cur_date = "081723"

library(readxl)
library(openxlsx)
library(tidyverse)
library(reshape2)
library(imputeLCMD)
library(vsn)

# reading in files
mRNA_df = data.frame(read_excel("Input/mRNA_Data_041423.xlsx", sheet = 2))

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.2     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.2     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.1     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: ‘reshape2’


The following object is masked from ‘package:tidyr’:

    smiths


Loading required package: tmvtnorm

Loading required package: mvtnorm

Loading required package: Matrix


Attaching package: ‘Matrix’


The following objects are masked from ‘package:tidyr’:

    expand, pack, unpack


Loading required package: stats4

Loading required package: gmm

Loading required package: sandwich

Loading required package: norm

This package has some ma

In [2]:
head(mRNA_df)

,Subject_No,Subject_ID,mRNA,Condensate,Burn_Condition,Concentration,Time_Point,ddCT,ddCT_pslog2
,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>
1,1,F_1,ALDH1A3,C,F,1,24,-6.6824749,4.480105
2,2,M_2,ALDH1A3,C,F,1,24,0.2560009,4.870661
3,3,M_3,ALDH1A3,C,F,1,24,0.4318636,4.879307
4,4,F_4,ALDH1A3,C,F,1,24,0.4193253,4.878692
5,6,M_6,ALDH1A3,C,F,1,24,-0.1550695,4.850246
6,1,F_1,ALDH1A3,C,F,1,72,-0.2119125,4.847400


Identifying the number of replicates for each sample condition.

In [3]:
count_mRNA_df = mRNA_df %>%
    group_by(mRNA, Subject_ID, Condensate, Burn_Condition, Concentration, Time_Point) %>%
    mutate(Duplicate_Count = n()) %>% 
    arrange(-Duplicate_Count, mRNA, Subject_No, Condensate, Burn_Condition, Concentration, Time_Point) 

head(count_mRNA_df)

Subject_No,Subject_ID,mRNA,Condensate,Burn_Condition,Concentration,Time_Point,ddCT,ddCT_pslog2,Duplicate_Count
<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<int>
6,M_6,HMOX1,PBS,PBS,NA,24,0.70848207,4.892803,6
6,M_6,HMOX1,PBS,PBS,NA,24,0.36651641,4.876100,6
6,M_6,HMOX1,PBS,PBS,NA,24,-0.09090465,4.853452,6
6,M_6,HMOX1,PBS,PBS,NA,24,-0.37654696,4.839126,6
6,M_6,HMOX1,PBS,PBS,NA,24,0.72283828,4.893500,6
6,M_6,HMOX1,PBS,PBS,NA,24,0.37135970,4.876338,6


In [4]:
replicate_count_df = count_mRNA_df %>%
    group_by(Duplicate_Count) %>%
    count() %>%
    arrange(-Duplicate_Count) 

replicate_count_df

Duplicate_Count,n
<int>,<int>
6,6
5,80
4,328
3,96
2,1340
1,2361


In [5]:
# creating a new column for the replicate number for each condition
replicate_vector = c()
for (i in 1:length(replicate_count_df$n)){
    # calculating number of times to repeat the replicate number from 1 to the count
    replicate_count = replicate_count_df$n[i]/replicate_count_df$Duplicate_Count[i]
    replicate_iteration = rep(1:replicate_count_df$Duplicate_Count[i], times = replicate_count)
    
    # saving iteration
    replicate_vector = c(replicate_vector, replicate_iteration)
    
}

# adding to df 
count_mRNA_df$Replicate = replicate_vector
mRNA_df = count_mRNA_df %>%
    select(-Duplicate_Count)

head(mRNA_df)

Subject_No,Subject_ID,mRNA,Condensate,Burn_Condition,Concentration,Time_Point,ddCT,ddCT_pslog2,Replicate
<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<int>
6,M_6,HMOX1,PBS,PBS,NA,24,0.70848207,4.892803,1
6,M_6,HMOX1,PBS,PBS,NA,24,0.36651641,4.876100,2
6,M_6,HMOX1,PBS,PBS,NA,24,-0.09090465,4.853452,3
6,M_6,HMOX1,PBS,PBS,NA,24,-0.37654696,4.839126,4
6,M_6,HMOX1,PBS,PBS,NA,24,0.72283828,4.893500,5
6,M_6,HMOX1,PBS,PBS,NA,24,0.37135970,4.876338,6


Take the average ddCT to get rid of the replicates.

In [6]:
avg_mRNA_df = mRNA_df %>%
    group_by(mRNA, Subject_ID, Condensate, Burn_Condition, Concentration, Time_Point) %>%
    mutate(Avg_ddCT = mean(ddCT)) %>% 
    select(-c("Replicate", "ddCT_pslog2", "ddCT")) %>%
    unique() %>%
    ungroup()

head(avg_mRNA_df)

Subject_No,Subject_ID,mRNA,Condensate,Burn_Condition,Concentration,Time_Point,Avg_ddCT
<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>
6,M_6,HMOX1,PBS,PBS,NA,24,0.2836241
5,F_5,ALDH3A1,PBS,PBS,NA,4,0.4320455
5,F_5,CXCL1,PBS,PBS,NA,4,0.6422482
5,F_5,CXCR1,PBS,PBS,NA,4,-12.4016243
5,F_5,GCLC,PBS,PBS,NA,4,0.9108458
5,F_5,GCLM,PBS,PBS,NA,4,0.3423809


# Variance Stabilizing Transformation
This makes it so that the variance doesn't depend on the mean.
https://book.stat420.org/transformations.html

In [7]:
# first converting from long to wide format, which is necessary for the vsn function
wider_avg_mRNA_df = avg_mRNA_df %>%
# creating sample ids
    unite("SampleID", c(colnames(avg_mRNA_df)[c(1,4:7)]), sep = "_") %>%
    select(-Subject_ID) %>%
    pivot_wider(names_from = "SampleID", values_from = "Avg_ddCT") %>%
    column_to_rownames(var = "mRNA")

head(wider_avg_mRNA_df)

,6_PBS_PBS_NA_24,5_PBS_PBS_NA_4,1_PBS_PBS_NA_4,2_PBS_PBS_NA_4,3_PBS_PBS_NA_4,4_PBS_PBS_NA_4,6_PBS_PBS_NA_4,6_PBS_PBS_NA_72,1_C_F_25_4,1_C_S_25_4,⋯,6_P_F_1_72,6_P_F_25_24,6_P_F_25_72,6_P_F_5_72,6_P_S_1_72,6_P_S_25_24,6_P_S_25_72,6_P_S_5_24,1_C_S_1_24,5_C_F_1_24
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
HMOX1,0.2836241,1.6220269,1.39023414,2.1102434,2.43685005,2.0919311,1.702591720,0.1381671,-2.33804668,-3.22246963,⋯,1.6515563,NA,0.22110064,0.7961182,0.8983407,-0.3778758,0.5217230,-0.1241551,0.2082109,-1.0969917
ALDH3A1,-0.4184588,0.4320455,0.27473008,0.1204694,0.07055185,-0.1803027,0.009163446,-1.0577533,0.16267528,0.41473799,⋯,-0.4625463,NA,-1.07376220,-0.9375553,-1.3269869,0.4181893,1.7985573,0.2135130,0.4982387,0.4459567
CXCL1,-0.7070283,0.6422482,0.09744932,0.7277660,0.35592590,0.1674529,0.394399876,0.8012238,0.03556161,1.09853908,⋯,-1.8990085,NA,-0.02792455,3.8624448,3.3316444,1.0462532,0.6576701,0.7632177,-0.1058011,-0.5811714
CXCR1,-13.6424811,-12.4016243,-13.70600229,-13.5821411,-13.41281279,-13.2795531,-12.613186667,-12.4136794,-12.90925169,-14.10350485,⋯,-15.0410890,NA,-13.34490982,-11.8789158,-12.5749978,-11.9184997,-12.7449949,-11.8478948,-15.5025570,-13.1482028
GCLC,0.1048251,0.9108458,0.90181502,0.6517629,0.74953458,0.6207409,0.653683036,0.1575451,-0.31896658,-0.02863545,⋯,0.3394122,NA,-0.82052995,-0.3441076,-1.1861843,0.1192901,0.8528297,0.1874478,-0.1877919,-0.6381514
GCLM,-0.4755476,0.3423809,0.44173568,0.2984414,0.36819982,0.2192093,0.007537574,-0.3875209,0.12643633,0.32477261,⋯,-0.3960734,NA,0.15394273,-0.3305644,-0.1075840,0.3011212,0.4951484,0.2571859,0.3863060,-0.1976982


In [8]:
normalized_matrix = justvsn(as.matrix(wider_avg_mRNA_df), minDataPointsPerStratum = 19)
head(normalized_matrix)

,6_PBS_PBS_NA_24,5_PBS_PBS_NA_4,1_PBS_PBS_NA_4,2_PBS_PBS_NA_4,3_PBS_PBS_NA_4,4_PBS_PBS_NA_4,6_PBS_PBS_NA_4,6_PBS_PBS_NA_72,1_C_F_25_4,1_C_S_25_4,⋯,6_P_F_1_72,6_P_F_25_24,6_P_F_25_72,6_P_F_5_72,6_P_S_1_72,6_P_S_25_24,6_P_S_25_72,6_P_S_5_24,1_C_S_1_24,5_C_F_1_24
HMOX1,3.912311,3.067190,3.843045,3.415457,3.234524,3.294681,3.820872,3.703908,-16.250897,0.7883619,⋯,2.629532,NA,-17.17265,-1.229183,-17.74044,2.638051,2.409027,-14.4664678,2.094107,1.833391
ALDH3A1,3.715852,2.605258,3.532961,2.692775,2.292604,2.443504,3.301868,3.237933,2.021796,3.6641076,⋯,1.228801,NA,-18.06649,-17.570614,-18.61131,3.065341,3.037754,-2.0847038,2.279381,2.853769
CXCL1,3.626645,2.698362,3.476909,2.953958,2.444569,2.610919,3.437396,3.909343,1.919807,3.8811991,⋯,-13.877329,NA,-17.39302,2.873008,-14.32618,3.331071,2.490578,0.3527938,1.862415,2.259730
CXCR1,-20.295235,-19.997376,-20.310570,-20.264881,-20.153403,-20.079270,-20.230977,-20.401478,-20.385166,-20.7294483,⋯,-20.368879,NA,-20.49317,-20.586011,-20.32967,-20.410924,-20.407928,-20.2729677,-20.551241,-20.421350
GCLC,3.864761,2.809204,3.715390,2.923745,2.630937,2.803508,3.521934,3.710346,1.590093,3.5035971,⋯,1.926795,NA,-17.92993,-16.870849,-18.56933,2.919338,2.600134,-2.4236255,1.795250,2.218369
GCLM,3.698636,2.563639,3.583843,2.774333,2.450760,2.634254,3.301268,3.517304,1.993447,3.6329506,⋯,1.301355,NA,-17.23548,-16.850149,-18.19948,3.009904,2.392533,-1.6508262,2.210667,2.510943


# QRILC Imputation

In [9]:
preimputed_df = data.frame(normalized_matrix)
imputed_QRILC_object = impute.QRILC(preimputed_df, tune.sigma = 0.1)
imputed_df = data.frame(imputed_QRILC_object[1]) %>%
    rownames_to_column(var = "mRNA") %>%
    pivot_longer(cols = 2:186, names_to = "Sample_ID", values_to = "Norm_ddCT") %>%
    separate(Sample_ID, into = c("Subject_No", "Condensate", "Burn_Condition", "Concentration", "Time_Point"))

# dataframes automatically add X to col names if it starts with a number so removing that here
imputed_df$Subject_No = as.numeric(gsub("X","", as.character(imputed_df$Subject_No)))

# adding back in subject ids
final_df = full_join(imputed_df, unique(avg_mRNA_df %>%
                        select(c("Subject_No", "Subject_ID"))))

# rearranging cols
final_df = final_df[,c(2,8,1,3:7)]
head(final_df)

Joining with `by = join_by(Subject_No)`


Subject_No,Subject_ID,mRNA,Condensate,Burn_Condition,Concentration,Time_Point,Norm_ddCT
<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
6,M_6,HMOX1,PBS,PBS,NA,24,3.912311
5,F_5,HMOX1,PBS,PBS,NA,4,3.067190
1,F_1,HMOX1,PBS,PBS,NA,4,3.843045
2,M_2,HMOX1,PBS,PBS,NA,4,3.415457
3,M_3,HMOX1,PBS,PBS,NA,4,3.234524
4,F_4,HMOX1,PBS,PBS,NA,4,3.294681


In [10]:
# original data with replicates
dim(mRNA_df)

# after averaging replicates
dim(avg_mRNA_df)

# adding in missing data for remaining records and imputing
dim(final_df)

[1] 4211   10

[1] 3162    8

[1] 3515    8

In [11]:
# exporting
write.xlsx(final_df, paste0(Output,"/", "Processed_mRNA_Data_", cur_date, ".xlsx"), rowNames = FALSE)